<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/multi_cls_sobert_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import wandb
wandb.login(key="ee23d2708eaedb89d7020a87e3ac4dea2b742261")
wandb.init(project="Multi output classification", entity="ai_expert", name="sobert_base")

wandb: Currently logged in as: ebinna-lee (ebinna-lee-x). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
from datasets import load_dataset

dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_training_samples.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_validation_samples.csv',
    split='train'
)
dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_test_samples.csv',
    split='train'
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [27]:
classes = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']
class2id = {'Algorithms' :0, 'Backend' : 1, 'Data Science' : 2, 'Databases' : 3, 'Dev Tools' : 4, 'Frontend' : 5, 'Mobile' :6, 'Systems' : 7, 'iOS/macOS' : 8}
id2class = {0 : 'Algorithms', 1: 'Backend', 2 : 'Data Science', 3 : 'Databases', 4 : 'Dev Tools', 5 : 'Frontend', 6 : 'Mobile', 7 : 'Systems', 8 :'iOS/macOS'}

In [28]:
from transformers import AutoTokenizer

model_path = 'mmukh/SOBertBase'
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
import ast

def preprocess_function(example):
   all_labels =  ast.literal_eval(example['Tags_new'])
   labels = [0. for i in range(len(classes))]
   for label in all_labels:
       label_id = class2id[label]
       labels[label_id] = 1.

   example = tokenizer(example['Title'] + ' ' + example['Body'], truncation=True, return_tensors="pt")
   example['labels'] = labels
   example['input_ids'] = example['input_ids'].squeeze(0)
   example['token_type_ids'] = example['token_type_ids'].squeeze(0)
   example['attention_mask'] = example['attention_mask'].squeeze(0)
   return example


In [30]:
tokenized_train_dataset = dataset_train.map(preprocess_function)
tokenized_valid_dataset = dataset_valid.map(preprocess_function)
tokenized_test_dataset = dataset_test.map(preprocess_function)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [31]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Title", "Body", "Tags_filtered", "Tags_list", "Tags_new", "Algorithms", "Backend", "Data Science", "Databases", "Dev Tools", "Frontend", "Mobile", "Systems", "iOS/macOS"])
tokenized_valid_dataset = tokenized_valid_dataset.remove_columns(["Title", "Body", "Tags_filtered", "Tags_list", "Tags_new", "Algorithms", "Backend", "Data Science", "Databases", "Dev Tools", "Frontend", "Mobile", "Systems", "iOS/macOS"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Title", "Body", "Tags_filtered", "Tags_list", "Tags_new", "Algorithms", "Backend", "Data Science", "Databases", "Dev Tools", "Frontend", "Mobile", "Systems", "iOS/macOS"])

In [32]:
from transformers import DataCollatorWithPadding
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import numpy as np

def sigmoid(x):
   return 1/(1 + np.exp(-x))


def compute_metrics(eval_pred):

  predictions, labels = eval_pred
  predictions = sigmoid(predictions)
  predictions = (predictions > 0.5).astype(int)
  accuracy = accuracy_score(labels, predictions)
  precision, recall, f1_score_result, _ = precision_recall_fscore_support(labels, predictions, average='micro')
  precision_macro, recall_macro, f1_score_result_macro, _ = precision_recall_fscore_support(labels, predictions, average='macro')
  precision_weighted, recall_weighted, f1_score_result_weighted, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

  flat_predictions = predictions.reshape(-1)
  flat_labels = labels.reshape(-1)
  flat_accuracy = accuracy_score(flat_labels, flat_predictions)

  return {
        'flat_accuracy' : flat_accuracy,
        'accuracy' : accuracy,
        'micro_precision': precision,
        'micro_recall': recall,
        'micro_f1': f1_score_result,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_score_result_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_score_result_weighted,
        }

In [34]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(classes),
   id2label=id2class, label2id=class2id,
  problem_type = "multi_label_classification")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at mmukh/SOBertBase and are newly initialized: ['bert.embeddings.token_type_embeddings.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(50048, 768, padding_idx=0)
      (position_embeddings): Embedding(2048, 768)
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [35]:
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=5,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   metric_for_best_model="micro_recall",
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_valid_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Flat Accuracy,Accuracy,Micro Precision,Micro Recall,Micro F1,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1
1,0.149600,0.113344,0.960752,0.717692,0.848308,0.863567,0.855869,0.746448,0.858812,0.784608,0.858086,0.863567,0.858753
2,0.107400,0.113787,0.959282,0.701846,0.821737,0.891690,0.855286,0.733073,0.883806,0.790102,0.830485,0.891690,0.858201
3,0.075400,0.118472,0.958701,0.697077,0.818340,0.891943,0.853558,0.736706,0.884055,0.792202,0.829192,0.891943,0.857207
4,0.057500,0.127632,0.958479,0.696462,0.817622,0.891056,0.852761,0.727518,0.885970,0.786418,0.826292,0.891056,0.855893
5,0.043300,0.132513,0.958821,0.699692,0.819974,0.890296,0.853690,0.723419,0.887059,0.783160,0.829439,0.890296,0.857058


TrainOutput(global_step=25000, training_loss=0.10023628662109375, metrics={'train_runtime': 5720.0963, 'train_samples_per_second': 17.482, 'train_steps_per_second': 4.371, 'total_flos': 4.131525019816022e+16, 'train_loss': 0.10023628662109375, 'epoch': 5.0})

In [37]:
trainer.evaluate()

{'eval_loss': 0.11847172677516937,
 'eval_flat_accuracy': 0.9587008547008548,
 'eval_accuracy': 0.697076923076923,
 'eval_micro_precision': 0.8183403068340307,
 'eval_micro_recall': 0.8919432480364834,
 'eval_micro_f1': 0.8535580070311553,
 'eval_macro_precision': 0.7367063227761468,
 'eval_macro_recall': 0.8840545868418339,
 'eval_macro_f1': 0.792201591670515,
 'eval_weighted_precision': 0.8291920183873162,
 'eval_weighted_recall': 0.8919432480364834,
 'eval_weighted_f1': 0.857206789121141,
 'eval_runtime': 106.6909,
 'eval_samples_per_second': 60.924,
 'eval_steps_per_second': 15.231,
 'epoch': 5.0}

In [38]:
trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_test_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [39]:
trainer.evaluate()

{'eval_loss': 0.12050133943557739,
 'eval_model_preparation_time': 0.0034,
 'eval_flat_accuracy': 0.9584074074074074,
 'eval_accuracy': 0.6926666666666667,
 'eval_micro_precision': 0.8149714001492167,
 'eval_micro_recall': 0.8963347921225383,
 'eval_micro_f1': 0.8537189006122183,
 'eval_macro_precision': 0.7247123902434043,
 'eval_macro_recall': 0.8941249458960517,
 'eval_macro_f1': 0.7849899279607487,
 'eval_weighted_precision': 0.8257411209564238,
 'eval_weighted_recall': 0.8963347921225383,
 'eval_weighted_f1': 0.857534990919172,
 'eval_runtime': 53.6964,
 'eval_samples_per_second': 55.87,
 'eval_steps_per_second': 13.967}

In [40]:
tuned_model_path = '/content/drive/My Drive/AiExpertCource/project/final/multi_cls_sobert_base'  # 모델과 토크나이저가 저장된 경로

In [41]:
model.save_pretrained(tuned_model_path)
tokenizer.save_pretrained(tuned_model_path)

('/content/drive/My Drive/AiExpertCource/project/final/multi_cls_sobert_base/tokenizer_config.json',
 '/content/drive/My Drive/AiExpertCource/project/final/multi_cls_sobert_base/special_tokens_map.json',
 '/content/drive/My Drive/AiExpertCource/project/final/multi_cls_sobert_base/tokenizer.json')

In [42]:
tuned_tokenizer = AutoTokenizer.from_pretrained(tuned_model_path)
tuned_model = AutoModelForSequenceClassification.from_pretrained(tuned_model_path)

In [43]:
# 예측 함수 정의
def predict(texts):
    # 텍스트를 토큰화하고 텐서로 변환
    inputs = tuned_tokenizer(texts, padding='max_length', truncation=True, max_length=2048, return_tensors='pt')

    # 모델을 사용해 예측 수행
    with torch.no_grad():
        outputs = tuned_model(**inputs)
        logits = outputs.logits

    # 시그모이드를 사용해 확률로 변환
    probabilities = torch.sigmoid(logits).numpy()

    # 각 클래스에 대해 threshold를 0.5로 설정하여 예측값(0 또는 1)으로 변환
    predictions = (probabilities > 0.5).astype(int)

    return predictions, probabilities

In [44]:
# 샘플 텍스트 입력 및 예측 수행
sample_texts = [
  "FFmpeg hevc rtsp stream decoding with frame loss" +
  "I need to decode my rtsp stream. When i`m using default HEVC and have some corrupted frames, my screen looks like this: Corrupted frame using HEVC, Grey image. But instead of this, I want to have a corrupted picture with pixels issue, like this: Same corrupted frame, but using HEVC_QSV."
]

# 예측 수행
predictions, probabilities = predict(sample_texts)

In [45]:
arr = np.array(predictions[0])

# 값이 1인 인덱스에 해당하는 매핑 가져오기
indices = np.where(arr == 1)[0]
indices

for idx in indices:
    tag = id2class[idx]
    print(tag)

Systems


In [46]:
wandb.finish()

eval/accuracy,█▄▂▂▃▂▁
eval/flat_accuracy,█▄▂▁▂▂▁
eval/loss,▁▁▃▆█▃▄
eval/macro_f1,▂▆█▄▁█▂
eval/macro_precision,█▄▅▂▁▅▁
eval/macro_recall,▁▆▆▆▇▆█
eval/micro_f1,█▇▃▁▃▃▃
eval/micro_precision,█▂▂▂▂▂▁
eval/micro_recall,▁▇▇▇▇▇█
eval/model_preparation_time,▁
eval/runtime,██████▁
